# Ежедневные данные по выявленным случаям COVID-19
### для городов/штатов: Москва, Лондон, Мадрид, Амстердам, Калифорния и Иллинойс

In [1]:
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
import requests as req
import json

## Москва

In [2]:
url_Moscow = 'https://russian-trade.com/coronavirus-russia/moskva/'
Moscow_data = pd.read_html(url_Moscow, flavor='html5lib', encoding='utf-8')[0]
Moscow_data

,Дата,Заражений,Смертей,Выздоровлений,Заражено на дату
0,20.12.2020,735900,10394,567743,157763
1,19.12.2020,728637,10317,562953,155367
2,18.12.2020,722178,10243,557028,154907
3,17.12.2020,715241,10171,551207,153863
4,16.12.2020,708530,10095,545430,153005
...,...,...,...,...,...
264,31.03.2020,1613,11,70,1532
265,30.03.2020,1226,6,28,1192
266,29.03.2020,1014,6,28,980
267,28.03.2020,817,4,18,795


In [3]:
file_Moscow = open('Moscow_covid_cases.csv', 'w')
Cases = [0]
for val in Moscow_data.sort_index(ascending=False).iterrows():
        date = val[1]['Дата']
        Cases.append(val[1]['Заражений'])
        div = val[1]['Заражений'] - Cases[-2]
        file_Moscow .write(f'{date} \t {div}\n')
file_Moscow.close()

## London

In [4]:
url_London = 'https://api.coronavirus.data.gov.uk/v2/data?areaType=region&areaCode=E12000007&metric=cumCasesBySpecimenDate&format=json&release=2020-12-19'

In [5]:
resp = req.get(url_London)
data_London = json.loads(resp.text)['body']

In [6]:
file_London = open('London_covid_cases.csv', 'w')
Cases = [0]
for key in data_London[::-1]:
    if key['cumCasesBySpecimenDate']:
        date = datetime.strptime(key['date'], '%Y-%m-%d')
        date = date.strftime('%d.%m.%Y')
        Cases.append(key['cumCasesBySpecimenDate'])
        div = key['cumCasesBySpecimenDate'] - Cases[-2]
        file_London.write(f'{date} \t {div}\n')
file_London.close()

# California

In [7]:
url_USA = 'https://github.com/nytimes/covid-19-data/blob/master/us-states.csv'

In [8]:
data_USA = pd.read_html(url_USA, flavor='html5lib')[0]

In [9]:
data_USA.head()

,0,1
0,NaN,"date,state,fips,cases,deaths"
1,NaN,"2020-01-21,Washington,53,1,0"
2,NaN,"2020-01-22,Washington,53,1,0"
3,NaN,"2020-01-23,Washington,53,1,0"
4,NaN,"2020-01-24,Illinois,17,1,0"


In [10]:
def write_csv_USA(state):
    name = str(state) + '_covid_cases.csv'
    file = open(name, 'w')
    Cases = [0]
    for key in data_USA[ data_USA[1].str.contains(str(state), case=False) ][1]:
        row_list = key.split(',')
        date = datetime.strptime(row_list[0], '%Y-%m-%d')
        date = date.strftime('%d.%m.%Y')
        CumulativeCases = int(row_list[3])
        Cases.append(CumulativeCases)
        div = CumulativeCases - Cases[-2]
        file.write(f'{date} \t {div}\n')
    file.close()

In [11]:
write_csv_USA('California')

In [12]:
write_csv_USA('Illinois')

# Madrid

In [13]:
url_Madrid = 'https://flo.uri.sh/visualisation/4440591/embed?auto=1'

In [14]:
resp = req.get(url_Madrid)
soup = BeautifulSoup(resp.text, 'html5lib')
script = soup.findAll('script')[4].string
cities_dict = script.split("_Flourish_data_column_names = ", 1)[-1].rsplit('\n', -1)[0][0:-1]
cities_dict = json.loads(cities_dict)

In [15]:
Madrid_column_position = cities_dict['data']['value'].index('Madrid')

In [18]:
Spain_data = script.split("_Flourish_data = ", 1)[-1].rsplit(';', -1)[0]
Spain_data = json.loads(Spain_data)

In [20]:
file_Madrid = open('Madrid_covid_cases.csv', 'w')
for value in Spain_data['data']:
    
    date = datetime.strptime(value['label'], '%d/%m/%Y')
    date = date.strftime('%d.%m.%Y')
    Cases = int(value['value'][Madrid_column_position])
    file_Madrid.write(f'{date} \t {Cases}\n')
    
file_Madrid.close()

# Amsterdam

In [21]:
url_Netherlands = 'https://coronadashboard.government.nl/gemeente/GM0363/positief-geteste-mensen'

In [22]:
resp = req.get(url_Netherlands)
soup = BeautifulSoup(resp.text, 'html5lib')
script = soup.findAll('script')[1].string
ams_dict = json.loads(script)

In [23]:
data_Netherlands = ams_dict['props']['pageProps']['data']['positive_tested_people']['values']

In [25]:
file_Amsterdam = open('Amsterdam_covid_cases.csv', 'w')
for val in data_Netherlands:
    date = datetime.fromtimestamp(val['date_of_report_unix'])
    date = date.strftime('%d.%m.%Y')
    Cases = int(val['infected_daily_total'])
    file_Amsterdam.write(f'{date} \t {Cases}\n')